In [1]:
from netgen.occ import *
from ngsolve import *
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
import numpy as np

## Computing £1 coin as a regular dodecagon of radius r mm:

 <img src="Figures/large_new_pound.webp" alt="isolated" width="200"/>

As with other objects, we define lists of material names, mur and sigma:

In [2]:
object_name = ['outer_frame', 'inner_cyln', 'plating']
sigma = [5.26E+06, 1.63E+07, 1E6]
mur = [1.15, 1, 100]

In [3]:
r = 23.43/2
t = 2.8

To define a dodegagon, we also need the side length. Since we know that there are 12 sides we know all the angles involved and can use the sine rule to obtain the chord length a.

In [4]:
a = r * np.sin(30 * np.pi/180) / np.sin(75 * np.pi/180)

Now that we have the radius and segment length, we can iterativly constuct a dodecagon of radius r and side length a.

### Building the dodecagon

WorkPlanes allows us to construct 2D shapes, in a similar way to turtle graphics, by moving and updating a cursor. 

In [5]:
wp = WorkPlane()
for i in range(12):
    wp.Line(a).Rotate(30)
face = wp.Face()
DrawGeo(face)

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [3.0320651133760297, 11.315…

BaseWebGuiScene

Now, we can turn this 2D shape into a 3D volume by extruding the face to the thickness of the coin: 

In [6]:
frame = face.Extrude(t)
DrawGeo(frame)

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [3.0320651133760297, 11.315…

BaseWebGuiScene

### Adding the interior
We now add the interior cylinder. To do this, we define a cylinder of radius r_inner and the same thickness as the rest of the coin:

In [7]:
r_inner = 7.4

In [8]:
cx = frame.center[0]
cy = frame.center[1]
inner_cyln = Cylinder(Pnt(cx,cy,0),Z, r=r_inner, h=t)

We can now subtract the inner cylinder from the rest of the coin to obtain only the outer region:

In [9]:
frame_outer = frame - inner_cyln
DrawGeo(frame_outer)

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [3.0320651133760297, 11.315…

BaseWebGuiScene

We now have everything we need to build the final coin, but before we join everything together we add object and material names, boundary condition names, and define the desired mesh density:

In [10]:
inner_cyln.mat(object_name[1])
frame_outer.mat(object_name[0])
inner_cyln.bc('inner')
inner_cyln.name = 'inner_cyln'
frame_outer.bc('frame_outer')
frame_outer.name = 'frame_outer'
inner_cyln.maxh = 1
frame_outer.maxh = 1

Finally, we can now build our final object:

In [11]:
coin = Glue([inner_cyln, frame_outer])
DrawGeo(coin)

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [3.0320651133760297, 11.315…

BaseWebGuiScene

and add a large outer non-conducting region:

In [12]:
box = Box(Pnt(-1000, -1000, -1000), Pnt(1000,1000,1000))
box.mat('air')
box.bc('outer')
box.maxh = 1000

### Generating Mesh
We have now defined the entire geometry of what we want to simulate and assigned the appropriate metadata. All that is left to do it to generate a mesh:

In [13]:
joined_object = Glue([coin, box])
geo = OCCGeometry(joined_object)
nmesh = geo.GenerateMesh()

The real £1 coin has a thin Nickel plating around the central cylinder. We can model this using a thin layer of prismatic elements by adding boundary layer elements of 50 $\mu$m thickness and tagged as Nickel to the inner part of the coin.

In [14]:
nmesh.BoundaryLayer(boundary=".*", thickness=[5e-5], material=object_name[2],
                           domains='inner_cyln', outside=False)

Finally, we save the mesh to disk.

In [15]:
nmesh.Save(r'../VolFiles/OCC_coin.vol')

We can also visualise the mesh using

In [16]:
ngmesh = Mesh(nmesh)
Draw(ngmesh, clipping=True)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2203-78-gade5a17a4', 'mesh_dim': 3, 'order2d':…

BaseWebGuiScene